In [4]:
%pip install tqdm


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import onnxruntime
from transformers import AutoTokenizer

In [2]:
# Load encoder model
encoder_session = onnxruntime.InferenceSession('oyto_t5_small_onnx/encoder_model.onnx')

# Load decoder model
decoder_session = onnxruntime.InferenceSession('oyto_t5_small_onnx/decoder_model.onnx')

In [18]:
# Function to preprocess input
def preprocess_input(input_text, tokenizer, max_source_length=1024):
    # Prefix to add before the source text
    prefix = "translate undiacritized Yoruba to diacritized Yoruba: "

    # Combine the prefix and input text
    input_text_with_prefix = prefix + input_text

    # Tokenize the input text
    model_inputs = tokenizer(
        [input_text_with_prefix],
        max_length=max_source_length,
        padding="max_length",
        truncation=True,
        return_tensors="np",  # Return NumPy arrays
    )

    return model_inputs

In [7]:
# Function to postprocess output
def postprocess_output(output_sequence, tokenizer):
    # Decode the generated output
    decoded_output = tokenizer.batch_decode(output_sequence, skip_special_tokens=True)[0]
    return decoded_output

In [19]:
# Tokenizer for the model
model_name_or_path = 'oyot5_small_unyo_dcyo_mix'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
# Preprocess input data
input_data = preprocess_input('mo fe jeun', tokenizer)

In [31]:
# Run inference on encoder
# encoded_context = encoder_session.run(None, {'input_ids': input_data['input_ids']})
encoded_context = encoder_session.run(
    None,
    {
        'input_ids': input_data['input_ids'],
        'attention_mask': input_data['attention_mask']
    }
)

In [32]:
print(encoded_context)
print(input_data)

[array([[[-0.2424832 , -0.1561737 ,  0.33006942, ...,  0.59456974,
         -0.40588298,  0.02734715],
        [-0.8927029 , -0.35662335,  0.2571173 , ...,  0.46694997,
         -0.628342  ,  0.59965885],
        [-0.00323499,  0.13572295,  0.00856398, ...,  0.0839715 ,
         -0.06416199, -0.14688414],
        ...,
        [ 0.3754469 ,  0.13625549, -0.00252384, ...,  0.15271264,
         -0.31701604,  0.09131931],
        [ 0.3754469 ,  0.13625549, -0.00252384, ...,  0.15271264,
         -0.31701604,  0.09131931],
        [ 0.3754469 ,  0.13625549, -0.00252384, ...,  0.15271264,
         -0.31701604,  0.09131931]]], dtype=float32)]
{'input_ids': array([[6819, 4997,    3, ...,    0,    0,    0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0]])}


In [35]:
# Run inference on decoder with the encoded context
# output_sequence = decoder_session.run(None, {'context_vector': encoded_context})
output_sequence = decoder_session.run(
    None,
    {
        'decoder_input_ids': encoded_context,
        'encoder_attention_mask': input_data['attention_mask'],
    }
)

ValueError: Required inputs (['input_ids', 'encoder_hidden_states']) are missing from input feed (['decoder_input_ids', 'encoder_attention_mask']).